# LLM Classifiers

**Goal:** Given a sentence as input, classify it as either a prediction or non-prediction.

In [1]:
import os
import sys

import pandas as pd

from tqdm import tqdm

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Add the parent directory to the system path
sys.path.append(os.path.join(notebook_dir, '../'))

# import log_files
from data_processing import DataProcessing
from classification_models import EvaluationMetric
from text_generation_models import TextGenerationModelFactory

In [2]:
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Load Data

In [3]:
print("======= LOAD DATA =======")

======= LOAD DATA =======


In [4]:
base_data_path = os.path.join(notebook_dir, '../data/')
combine_data_path = os.path.join(base_data_path, 'combined_generated_fin_phrase_bank')
X_test_set_path = os.path.join(combine_data_path, 'x_test_set-v1.csv')
y_test_set_path = os.path.join(combine_data_path, 'y_test_set-v1.csv')

In [5]:
X_test_df = DataProcessing.load_from_file(X_test_set_path, 'csv', sep=',')
X_test_df.drop(columns=['Unnamed: 0'], inplace=True)
X_test_df.head(7)

,Base Sentence,Sentence Label,Embedding,Normalized Embeddings
0,"Dr. Maria Rodriguez, a renowned climate scientist, noted on 2025-02-14 that the wind speed in Chicago fell.",0,[-5.33820353e-02 2.96053916e-01 4.16866988e-02 -1.40752643e-03\n 6.82408139e-02 7.03541860e-02 5.54416962e-02 -4.47977930e-02\n -1.14564508e-01 1.61363328e+00 -2.44681358e-01 6.63914764e-03\n -5.85275004e-03 -4.76306081e-02 -2.46703267e-01 -4.53357548e-02\n 2.61321273e-02 1.04844749e+00 -5.91918267e-02 -1.12772949e-01\n 9.06320885e-02 7.04300851e-02 3.07557527e-02 -1.40855715e-01\n 6.73192963e-02 -3.19030066e-03 -1.22142859e-01 1.88190360e-02\n 1.03679232e-01 5.15614264e-02 5.91920912e-02 1.00362174e-01\n -7.44822100e-02 -3.64652388e-02 9.92875174e-02 -4.76585925e-02\n 5.13628684e-02 1.03396410e-02 -8.16966295e-02 -5.59681691e-02\n 1.17678136e-01 2.33407039e-02 1.11587368e-01 -4.07175086e-02\n 2.27868054e-02 5.68023510e-02 -1.29113361e-01 -8.67207572e-02\n 8....,[ 0.3908352 0.88384986 0.54582196 0.5529683 0.20168522 1.4207762\n 1.1209849 -0.35026643 -2.0581949 -0.5683589 0.43492547 -0.28378505\n -1.2241656 -0.38047555 -2.0140388 -0.12880665 0.59478503 -0.2763546\n 0.88904005 -1.2983011 1.117075 0.3034963 0.53709686 -0.84479505\n 0.5376486 -0.9335876 0.04107084 -0.08168354 1.4058938 -0.2758865\n 0.98090166 1.413457 -1.1836389 -1.6302584 0.88330925 -0.423647\n 0.94221413 -0.5136003 -0.9248387 -0.41756865 1.8539761 -0.34789968\n 0.43764463 -0.03285579 -0.05317093 0.83162355 -0.9312643 -0.530558\n 0.6130175 0.3930604 -0.19065279 2.0767083 -0.67761785 0.48515055\n -0.41890967 0.5868992 0.44187045 0.7070218 1.0301542 -1.2024007\n -0.19246452 0.2535662 0.4490124 0.31755912 -0.5983049 -1....
1,"We succeeded in increasing our market share of sold apartment '' , comments Mr Kari Kauniskangas , Head of YIT International Construction Services .",0,[-4.71338071e-02 2.51612782e-01 2.61100605e-02 -1.26865417e-01\n 1.76884234e-01 6.63294748e-04 5.20298518e-02 -3.61273251e-02\n 2.52827350e-02 1.79729569e+00 -2.72558719e-01 -1.54113322e-02\n 1.13873892e-01 -1.15707785e-01 -1.11250550e-01 -4.33727466e-02\n -2.12675184e-02 9.76092160e-01 -1.39500961e-01 6.79444596e-02\n 4.63146679e-02 1.58941254e-01 2.58127805e-02 -1.83992460e-02\n 7.52350911e-02 2.80956198e-02 -5.96148474e-03 4.35078591e-02\n 1.56810004e-02 1.25586882e-01 9.25043598e-02 -2.61787921e-02\n -4.60829176e-02 8.23754594e-02 3.49306278e-02 -1.32550314e-01\n 4.81028110e-03 3.48891094e-02 1.02087855e-04 1.98335554e-02\n -1.02561712e-01 -6.57700896e-02 6.17823340e-02 -4.99959737e-02\n -1.31786028e-02 1.17884390e-01 1.76247712e-02 1.19141666e-02\n 1....,[ 0.46643826 0.28396165 0.3074835 -1.4569969 1.5413463 0.4508572\n 1.0626299 -0.24701537 -0.13232419 0.04329398 0.10356648 -0.56628853\n 0.6285996 -1.3918105 -0.6639517 -0.09610927 -0.18943849 -0.70547533\n -0.33030978 1.5931151 0.38704258 1.5418671 0.4651784 0.84381676\n 0.6579603 -0.4899005 1.6089592 0.29092404 0.06360818 0.76065934\n 1.5474954 -0.41184446 -0.7293663 0.01019586 -0.1011791 -1.8380105\n 0.29979753 -0.1342902 0.24613483 0.5835421 -1.8709322 -1.7238133\n -0.18806958 -0.17735313 -0.5564406 1.8359636 1.1352255 0.6281795\n 0.8216415 -0.28341526 0.23557287 -0.6368683 -0.16734943 1.7343459\n 0.63090867 0.507011 0.01599794 -0.76570714 0.40616304 0.9434384\n 0.26858085 -0.6297688 -0.63668746 -1.6477914 0.96522474 ...
2,"The Centers for Disease Control and Prevention reported that on 4/12/2032, the prevalence of obesity among children increased.",0,[-1.51246399e-01 2.89643884e-01 -6.67415932e-02 -1.25964195e-01\n -1.45232335e-01 -1.85876876e-01 9.91560966e-02 -1.26654934e-02\n 1.91853829e-02 2.25272012e+00 -2.79949337e-01 -1.65273547e-02\n 1.05223402e-01 1.76420361e-02 -3.08898333e-02 -1.08919702e-01\n 6.20932877e-02 1.31970799e+00 -2.89755166e-01 -2.24788003e-02\n -5.98008223e-02 -7.73710106e-03 -1.55985981e-01 -1.21320404e-01\n 5.34137487e-02 -3.02765016e-02 -3.34187560e-02 2.55857743e-02\n 5.20368591e-02 -7.90180359e-03 -2.90252268e-02 4.36594449e-02\n 4.220

In [6]:
y_test_df = DataProcessing.load_from_file(y_test_set_path, 'csv', sep=',')
y_test_df.drop(columns=['Unnamed: 0'], inplace=True)
# print(f"\t{y_test_df.head(7)}")

In [7]:
print(f"\tShape: {X_test_df.shape}, \nSubset of Data:{X_test_df.head(3)}")
# df.shape, df.head(3)

print(f"\tShape: {y_test_df.shape}, \nSubset of Data:{y_test_df.head(3)}")
# df.shape, df.head(3)

	Shape: (565, 4), 
Subset of Data:                                                                                                                                          Base Sentence  \
0                                           Dr. Maria Rodriguez, a renowned climate scientist, noted on 2025-02-14 that the wind speed in Chicago fell.   
1  We succeeded in increasing our market share of sold apartment '' , comments Mr Kari Kauniskangas , Head of YIT International Construction Services .   
2                        The Centers for Disease Control and Prevention reported that on 4/12/2032, the prevalence of obesity among children increased.   

   Sentence Label  \
0               0   
1               0   
2               0   

                                                                                                                                                                                                                                                                    

In [8]:
# for idx, row in X_test_df.iterrows():
#     text = row['Base Sentence']
#     label = row['Sentence Label']
#     embedding = row['Embedding']
#     print(type(embedding))
#     norm_embedding = row['Normalized Embeddings']
#     if idx < 7:
#         print(f"{idx}\n Sentence: {text}\n Label: {label}\n Embeddings Shape: {embedding}\n\t Embeddings Subset [:6]: {embedding[:6]} \n Norm Embeddings: {norm_embedding.shape}, \n\tNorm Embeddings Subset [:6]: {norm_embedding[:6]}")
# print(f"\tShape: {y_test_df.shape}, \nSubset of Data:{y_test_df.head(7)}")

## Load Prompt

In [9]:
# prediction_properties = PredictionProperties.get_prediction_properties()
# prediction_requirements = PredictionProperties.get_requirements()
# system_identity_prompt = "You are an expert at identifying specific types of sentences by knowing the sentence format."
# prediction_examples_prompt = """Some examples of predictions in the PhraseBank dataset are
#     1. According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .
#     2. According to the company 's updated strategy for the years 2009-2012 , Basware targets a long-term net sales growth in the range of 20 % -40 % with an operating profit margin of 10 % -20 % of net sales .
#     3. Its board of directors will propose a dividend of EUR0 .12 per share for 2010 , up from the EUR0 .08 per share paid in 2009 .
# """
# non_prediction_examples_prompt = """Some examples of non-predictions in the PPhraseBank dataset are
#     1. Net sales increased to EUR193 .3 m from EUR179 .9 m and pretax profit rose by 34.2 % to EUR43 .1 m. ( EUR1 = USD1 .4 )
#     2. Net sales surged by 18.5 % to EUR167 .8 m. Teleste said that EUR20 .4 m , or 12.2 % , of the sales came from the acquisitions made in 2009 .
#     3. STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .
# """
# # goal_prompt = "Given the above, identify the prediction."

# base_prompt = f"""{system_identity_prompt} The sentence format is based on: 
    
#     {prediction_properties}
#     Enforce: {prediction_requirements}
#     Know: {prediction_examples_prompt}
#     Know: {non_prediction_examples_prompt}

# """
# base_prompt

In [10]:
prompt_1 = """ 

Role: 
You are a linguist expert. You are acting as a prediction detector. Your task is to identify if a given sentence is a prediction about the future.

Background:
A prediction is a statement about what someone thinks will happen in the future.
Examples of predictions:
- "It will rain tomorrow." (Yes)
- "The stock market is expected to rise next quarter." (Yes)
- "I am going to the store." (No)
- “Lakers will win the championship. ”(Yes)

A prediction may contain: source, target, date, outcome.
"""

## Models

In [11]:
tgmf = TextGenerationModelFactory()

# Option 1: Specific models
models = tgmf.create_instances(['llama-3.1-8b-instant', 'llama-3.3-70b-versatile', 'llama-3.3-70b-instruct'])

# Option 2: All Groq models
# models = tgmf.create_instances(tgmf.get_groq_model_names())

# Option 3: All NaviGator models
# models = tgmf.create_instances(tgmf.get_navigator_model_names())

# Option 4: All available models
# models = tgmf.create_instances()

# Option 5: Mix and match
# custom_models = ['llama-3.1-70b-instruct', 'mistral-small-3.1', 'llama-3.1-8b-instant']
# models = tgmf.create_instances(custom_models)
models

In [12]:
import json
import re

def parse_json_response(response):
    """Parse JSON response from LLM to extract label and reasoning"""
    try:
        # Extract JSON if there's extra text
        json_match = re.search(r'\{.*\}', response, re.DOTALL)
        if json_match:
            data = json.loads(json_match.group())
            return data.get('label'), data.get('reasoning')
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        return None, None

In [13]:
def llm_certifier(data: str, base_prompt: str, model):
    
        prompt = f""" Given this: {base_prompt}. Also given the sentence '{data}', your task is to analyze the sentence and determine if it is a prediction. If prediction, generate label as 1 and if non-prediction generate label as 0.
        Respond ONLY with valid JSON in this exact format:
        {{"label": 0, "reasoning": "your explanation here"}}
        Examples:
        - "It will rain tomorrow." → {{"label": 1, "reasoning": "Contains the future tense words 'will' and 'tomorrow'"}}
        - "The stock market is expected to rise next quarter." → {{"label": 1, "reasoning": "Contains future tense words 'is expected'"}}
        - "I am going to the store." → {{"label": 0, "reasoning": "Does not contain a future tense word"}}
        - "Lakers will win the championship." → {{"label": 1, "reasoning": "Contains the future tense word 'will'"}}
        """
        idx = 1
        if idx == 1:
            #   print(f"\tPrompt: {prompt}")
              idx = idx + 1
        input_prompt = model.user(prompt)
        raw_text_llm_generation = model.chat_completion([input_prompt])
        
        # Parse the JSON response
        label, reasoning = parse_json_response(raw_text_llm_generation)
        
        return raw_text_llm_generation, label, reasoning

In [14]:
print("======= PROMPT + MODEL -> LABEL and REASONING =======")

======= PROMPT + MODEL -> LABEL and REASONING =======


In [ ]:
    # content : meta :: text : meta_data
results = []
for idx, row in X_test_df.iterrows():
    text = row['Base Sentence']
    print(f"{idx} --- Sentence: {text}")
    for model in models:
        # print(model.__name__())
        raw_response, llm_label, llm_reasoning = llm_certifier(text, prompt_1, model)
        print(f"\tModel: {model.__name__()}\n\t\tLabel:', {llm_label}\n\t\tReasoning:', {llm_reasoning}")
        result = (text, raw_response, llm_label, llm_reasoning, model.__name__())
        results.append(result)


0 --- Sentence: Dr. Maria Rodriguez, a renowned climate scientist, noted on 2025-02-14 that the wind speed in Chicago fell.
	Model: llama-3.1-8b-instant
		Label:', 0
		Reasoning:', The sentence describes a past event, 'the wind speed in Chicago fell', and does not contain any future tense words or phrases.
	Model: llama-3.3-70b-versatile
		Label:', 0
		Reasoning:', The sentence is in the past tense, as indicated by the verb 'noted' and the date '2025-02-14', and it describes a past event, 'the wind speed in Chicago fell', without any reference to a future event or outcome.
	Model: llama-3.3-70b-instruct
		Label:', 0
		Reasoning:', The sentence is in the past tense, indicated by the word 'noted' and the specific date '2025-02-14', and it describes a completed action, 'the wind speed in Chicago fell', which suggests a statement about a past event rather than a prediction about the future.
1 --- Sentence: We succeeded in increasing our market share of sold apartment '' , comments Mr Kari 

In [ ]:
results

In [ ]:
# groupby text

In [ ]:
results_with_llm_label_df = pd.DataFrame(results, columns=['text', 'raw_response', 'llm_label', 'llm_reasoning', 'llm_name'])
results_with_llm_label_df

In [ ]:
y_test_df.rename(columns={'Sentence Label' : 'Actual Label'}, inplace=True)

In [ ]:
def get_llm_labels(df, model_name):
    filt_llama = (df['llm_name'] == model_name)
    filt_df = df[filt_llama]
    return filt_df['llm_label']


test_and_models_df = pd.concat([X_test_df.loc[:, :], y_test_df], axis=1)
# test_and_models_df.columns = ['Sentence', 'Actual Label']
for model in models:
    print(model.__name__())
    model_labels = get_llm_labels(results_with_llm_label_df, model.__name__())
    test_and_models_df[model.__name__()] = model_labels.to_numpy().ravel()
test_and_models_df

## Save Output

In [ ]:
# save_path = os.path.join(combine_data_path )
DataProcessing.save_to_file(test_and_models_df, combine_data_path, 'llm_classifiers', '.csv')

## Evaluation

In [ ]:
print("======= EVALUATION/RESULTS =======")

In [ ]:
get_metrics = EvaluationMetric()
get_metrics

In [ ]:
actual_label = test_and_models_df['Actual Label'].values
for ml_model in models:
    ml_model_name = ml_model.__name__()
    print(f"Actual Label:\t\t{actual_label}")
    ml_model_predictions = test_and_models_df[ml_model_name].values
    print(f"{ml_model_name}:\t\t{ml_model_predictions}")
    get_metrics.eval_classification_report(y_test_df, ml_model_predictions)